In [0]:
print("please work")

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, TimestampType
 
customers_schema = StructType(fields=[StructField("customer_id", IntegerType()),
                                     StructField("customer_name", StringType()),
                                     StructField("date_of_birth", DateType()),
                                     StructField("telephone", StringType()),
                                     StructField("email", StringType()),
                                     StructField("member_since", DateType()),
                                     StructField("created_timestamp", TimestampType())
                                    ]
                              )
 

In [0]:
customers_df = (
    spark.readStream
    .format("csv")
    .schema(customers_schema)
    .load('/Volumes/workspace/default/databricks-1/streaming/')
)

In [0]:
from pyspark.sql.functions import col, current_timestamp
 
customers_transformed_df = (
    customers_df.withColumn("file_path",col("_metadata.file_path"))
    .withColumn("ingestion_date",current_timestamp())
    )

Transformed data stream to Delta Table

In [0]:
streaming_query = (
    customers_transformed_df.writeStream
    .format("delta")
    .option("checkpointLocation", "/Volumes/workspace/default/databricks-1/checkpoints/customers")
    .trigger(once=True)
    .toTable("swathy_catalog.swathy_schema.customers_stream")
)

In [0]:
%sql
SELECT count(*)
FROM swathy_catalog.swathy_schema.customers_stream